In [211]:
import os
import pandas as pd
from pathlib import Path
import openpyxl
import tabula

In [3]:
download_path = "./Download"
file_path = (f"{download_path}/2017_1.xlsx")
df = pd.read_excel(file_path, engine='openpyxl')
df.tail()

,Počty podnikatelů dle občanství podnikajících v České republice,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
161,Jemen,887,YE,YEM,10,1,0,2,0,3,1,0,0,1,3,1,1,0,23
162,Srbsko a Černá Hora,891,CS,SCG,259,54,16,15,10,24,11,5,1,2,21,8,15,7,448
163,Zambie,894,ZM,ZMB,1,2,0,0,0,0,0,0,0,0,0,0,0,0,3
164,Součet,SUM1,NaN,NaN,328403,257904,123176,108217,55388,131755,91623,106193,90211,85226,218492,106506,187649,108161,1998904
165,Z toho cizinci,SUM2,NaN,NaN,31712,9528,2852,5507,4934,7874,2598,2546,2160,1532,7097,2118,4124,1814,86396


In [20]:
df = df[3:]
df.head()

,Počty podnikatelů dle občanství podnikajících v České republice,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
3,Občanství,Kód,Kód zn.,Kód zn.,Hlavní město Praha,Středočeský kraj,Jihočeský kraj,Plzeňský kraj,Karlovarský kraj,Ústecký kraj,Liberecký kraj,Královéhradecký kraj,Pardubický kraj,Kraj Vysočina,Jihomoravský kraj,Olomoucký kraj,Moravskoslezský kraj,Zlínský kraj,Součet
4,Afghánistán,004,AF,AFG,16,7,0,0,1,6,2,0,0,1,13,4,2,0,52
5,Albánie,008,AL,ALB,13,8,4,1,1,3,1,2,1,1,2,1,1,4,43
6,Alžírsko,012,DZ,DZA,94,22,3,0,1,7,2,1,1,1,9,2,2,0,145
7,Angola,024,AO,AGO,9,2,2,0,0,3,2,1,1,0,6,3,2,2,33


In [205]:
# Automatizace převodu celé složky .XLSX na správné .CSV
# Otevře, převede na .csv, odebere 4 první řádky, 
# resetuje index, uloží jako .csv

download_path = "./Download"
export_path = "./Preprocessed_CSV"


folder_path = Path(download_path)

processed = 0
all_dataframes = []

# Prochází všechny soubory ve složce
for file in folder_path.iterdir():
    if file.is_file() and not file.name.startswith('.'):  # Kontrola, zda jde o soubor

        # Definice nových názvů
        file_path = (f"{download_path}/{file.name}")  # Proměnná filepath
        newname = (file.name).replace(".xlsx", "")  # Odstranění .xlsx ze starého názvu
        csv_path = (f"{export_path}/{newname}.csv")  # Nový název pro uložení

        # Otevře excel v pandasu
        df = pd.read_excel(file_path, engine='openpyxl')  # Načíst excel jako pandas df

        # Stav
        processed += 1
        print(f"{processed} Soubor {csv_path} úspěšně převeden", end=" ")

        # Zahodíme 6 zbytečných řádků, poslední sloupec + resetujeme index
        df = df[3:].reset_index(drop=True) # Odstraní první 4 řádky
        df = df.iloc[:,:-1]  # Odstraní poslední sloupec
        df = df[:-2]  # Odstraní poslední 2 řádky
        df.columns = df.iloc[0]  # Nastaví 5. řádek (který je teď první po odstranění) jako názvy sloupců
        df = df[1:]  # Odstraní řádek použitý jako názvy sloupců
        df = df.set_index("Občanství")  # Nastaví sloupec "Občanství" jako index
        df = df.sort_values(by=df.columns[0], ascending=True)  # Utřídí názvy řad abecedně


        # Definice proměnných q a rok
        q = newname[-1]
        rok = newname[-6:-2]

        # Přidání suffixu do názvu sloupců
        for i, col in enumerate(df.columns):
            if i >= 3:  # Začíná na 4. sloupci
                df.rename(columns={col: f"{col} {rok} Q{q}"}, inplace=True)

        # Reset
        #df.reset_index(drop=True, inplace=True)  #  Pro jistotu ještě jednou index reset

        # Append the processed DataFrame to the list for merging
        all_dataframes.append(df)

        # Uloží jako CSV
        df.to_csv(csv_path, index=True)
        shape = df.shape
        print(f"+ uložen + přidán df do listu. Shape: {shape}", end="\n")


print(f"Ve složce je {processed} souborů.")

# Část s mergem v další buňce


1 Soubor ./Preprocessed_CSV/2024_3.csv úspěšně převeden + uložen + přidán df do listu. Shape: (252, 17)
2 Soubor ./Preprocessed_CSV/2019_1.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
3 Soubor ./Preprocessed_CSV/2016_4.csv úspěšně převeden + uložen + přidán df do listu. Shape: (161, 17)
4 Soubor ./Preprocessed_CSV/2017_1.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
5 Soubor ./Preprocessed_CSV/2024_2.csv úspěšně převeden + uložen + přidán df do listu. Shape: (252, 17)
6 Soubor ./Preprocessed_CSV/2018_4.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
7 Soubor ./Preprocessed_CSV/2018_3.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
8 Soubor ./Preprocessed_CSV/2018_2.csv úspěšně převeden + uložen + přidán df do listu. Shape: (161, 17)
9 Soubor ./Preprocessed_CSV/2017_4.csv úspěšně převeden + uložen + přidán df do listu. Shape: (161, 17)
10 Soubor ./Preprocessed_CSV/2018_1.csv úspěšně převeden + ulože

In [187]:
'''
# Zjistíme, kde se děje problém s prázdnými poli v prvních 4 sloupcích
# prohledává už spojený soubor merged_df

nan_rows = merged_df.iloc[:, :4].isna().any(axis=1)
nan_indexes = merged_df[nan_rows].index.tolist()
print("Indexy s prázdným polem v jednom z prvních 4 sloupců:")
print(nan_indexes)
'''


Index s prázdným polem v jednom z prvních 4 sloupců:
['Velká Británie', 'Nizozemsko', 'Makedonie', 'Demokratická republika Kongo', 'Namibie', 'Curacao']


In [206]:
# Merge všech CSV do jednoho
# Vyplnit 0 na místo všech chybějících hodnot
# Ověřit, jestli nějaký řádek nemá všude 0


# Loop through the list of DataFrames and merge them progressively
merged_df = all_dataframes[0]  # Start with the first DataFrame

# Merge progressively
for df in all_dataframes[1:]:
    # Merge only the columns that start from the 5th column onwards
    merged_df = merged_df.merge(df.iloc[:, 4:], how="outer", left_index=True, right_index=True)

# Doplnění 0 a převod typů na int
merged_df = merged_df.fillna(0)

# Najít jestli nějaké země mají všude 0
rows_all_zero = merged_df.iloc[:, 4:][merged_df.iloc[:, 4:].eq(0).all(axis=1)]  # Řádky s hodnotou 0 na všech pozicích
rows_to_delete = rows_all_zero.index.tolist()
merged_df.drop(rows_to_delete, inplace=True)

if rows_to_delete:
    print("Smazány řádky: ", end="")
    for i in rows_to_delete:
        print(i, end=", ")
else:
    print("Nenalezeny žádné řádky s 0")

# Optional: Save the merged DataFrame to a new CSV file
merged_csv_path = f"{export_path}/merged_dataset_2.csv"
merged_df.to_csv(merged_csv_path, index=True)
print(f"\nVšechny soubory sloučeny do: {merged_csv_path}", end="\n\n\n")


Smazány řádky: Americká Samoa, Americké Panenské ostrovy, Andorra, Anguilla, Antarktida, Antigua a Barbuda, Aruba, Bahamy, Bahrajn, Barbados, Belize, Bermudy, Bhútán, Bonaire, Svatý Eustach a Saba, Bouvetův ostrov, Britské indickooceánské území, Brunej, Cookovy ostrovy, Curaçao, Dominika, Džibutsko, Eritrea, Faerské ostrovy, Falklandy (Malvíny), Francouzská Polynésie, Francouzská jižní a antarktická území, Gabon, Gibraltar, Grenada, Grónsko, Guadeloupe, Guam, Guinea-Bissau, Guyana, Haiti, Heardův ostrov a MacDonaldovy ostrovy, Jersey, Jižní Georgie a Jižní Sandwichovy ostrovy, Jižní Súdán, Kajmanské ostrovy, Katar, Kiribati, Kokosové (Keelingovy) ostrovy, Komory, Lesotho, Macao, Malawi, Man, Marshallovy ostrovy, Martinik, Mayotte, Menší odlehlé ostrovy USA, Mikronésie, Montserrat, Mosambik, Nauru, Niue, Nizozemské Antily, Norfolk, Nová Kaledonie, Omán, Palau, Papua Nová Guinea, Pitcairn, Portoriko, Rovníková Guinea, Réunion, Saint Pierre a Miquelon, Samoa, San Marino, Severní Mariany, 

/var/folders/dj/72cg9gkd69z_1134dyl3zsqw0000gn/T/ipykernel_2721/3196127731.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.fillna(0)


In [207]:
merged_df.shape

(165, 212)

In [208]:
# Sloupec "Součet" na konci souboru
# sečte všechny ostatní číselné sloupce, abychom je mohli seřadit
# efektivní metoda, kdy se nefragmentuje výsledný df

sum_columns = merged_df.iloc[:, 4:].sum(axis=1).astype(int)

# Add the "Součet" column in one operation
merged_df = pd.concat([merged_df, sum_columns.rename("Součet")], axis=1)

# Sort the DataFrame by the "Součet" column in descending order
merged_df = merged_df.sort_values(by="Součet", ascending=False)

# Save to CSV
merged_df.to_csv(merged_csv_path, index=True)


In [209]:
merged_df["Součet"]

Občanství
Česko           25661897
Vietnam           275302
Ukrajina          217936
Slovensko         183716
Německo            42049
                  ...   
Kapverdy               2
Fidži                  2
Paraguay               1
Sierra Leone           1
Surinam                1
Name: Součet, Length: 165, dtype: int64

In [273]:
''''
JEDNORÁZOVÉ SPRAVENÍ SOUBORU 2016_2.csv

file_path = 'Down/2016_2.csv'  # Replace with your file path
df = pd.read_csv(file_path, sep=';')

# Save the file with commas as the delimiter
output_path = 'fixed_file.csv'  # Replace with your desired output path
df.to_csv(output_path, index=False, sep=',')
'''

In [ ]:
'''
------------------------------ THE END ---------------------------------
'''